In [5]:
import numpy as np
import pandas as pd

In [6]:
data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

data = np.array(data)

In [7]:
m , n =data.shape 

m,n

data_train = data[0:m].T
x = data_train[ 1 :n ]
x = x/225

y = data_train [0]


x_train = x[ : , : 38000]
y_train = y[:38000]

x_test = x[ : , 38000:]
y_test = y[38000:]

In [26]:
def init_params ():

    w1 = np.random.rand(128,784) - 0.5
    b1 = np.random.rand(128,1) - 0.5

    w2 = np.random.rand(64,128) - 0.5
    b2 = np.random.rand(64,1) - 0.5

    w3 = np.random.rand(32,64) - 0.5
    b3 = np.random.rand(32,1) - 0.5

    w4 = np.random.rand(10,32) - 0.5
    b4 = np.random.rand(10,1) - 0.5
    
    return w1, b1, w2, b2, w3, b3, w4, b4



def Relu(z):
    return  np.maximum(0,z)

def soft_max(z):
    
    exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))  
    return exp_z / np.sum(exp_z, axis=0, keepdims=True)


def forward_prop ( w1, b1, w2, b2, w3, b3, w4, b4, x):

    z1 = w1.dot(x)+ b1
    a1 = Relu(z1)

    z2 = w2.dot(a1)+ b2
    a2 = Relu(z2)

    z3 = w3.dot(a2)+ b3
    a3 = Relu(z3)

    z4 = w4.dot(a3)+ b4
    a4 = soft_max(z4)

    return z1, a1, z2, a2, z3, a3, z4, a4 

def relu_deriv(z):
    return (z > 0).astype(float)

def one_hot(y):

    one_hot_y = np.zeros((y.size , y.max() +1))
    one_hot_y[np.arange(y.size ) , y] = 1
    one_hot_y = one_hot_y.T

    return one_hot_y


def back_prop(z1, a1, z2, a2, z3, a3, z4, a4, w4, w3, w2, y, x ):

    m = y.size
    one_hot_y = one_hot(y)

    dz4 = a4- one_hot_y
    dw4 = 1/m * dz4.dot(a3.T)
    db4 = 1/m * np.sum(dz4, axis=1, keepdims = True)

    dz3 = w4.T.dot(dz4) * relu_deriv(z3)
    dw3 = 1/m * dz3.dot(a2.T)
    db3 = 1/m * np.sum ( dz3, axis = 1, keepdims = True)

    dz2 =  w3.T.dot(dz3) * relu_deriv(z2)
    dw2 = 1/m * dz2.dot(a1.T)
    db2 = 1/m * np.sum ( dz2, axis = 1, keepdims = True)

    dz1 =  w2.T.dot(dz2)  * relu_deriv(z1)
    dw1 = 1/m * dz1.dot(x.T)
    db1 = 1/m * np.sum(dz1, axis = 1, keepdims = True)

    return (dw1, db1, dw2, db2, dw3, db3, dw4, db4)

def update_params(w1, w2, w3, w4, b1, b2, b3, b4, dw1, db1, dw2, db2, dw3, db3, dw4, db4, alpha):

    w1 = w1- alpha*dw1
    b1 = b1- alpha*db1

    w2 = w2- alpha*dw2
    b2 = b2- alpha*db2

    w3 = w3- alpha*dw3
    b3 = b3- alpha*db3

    w4 = w4- alpha*dw4
    b4 = b4- alpha*db4

    return w1, b1, w2, b2, w3, b3, w4, b4 


def grad_descent( w1, b1, w2, b2, w3, b3, w4, b4, x , y , iterations , alpha):
    

    for i in range(iterations):
        z1, a1, z2, a2, z3, a3, z4, a4 = forward_prop(w1, b1, w2, b2, w3, b3, w4, b4, x)

        dw1, db1, dw2, db2, dw3, db3, dw4, db4 = back_prop(
            z1, a1, z2, a2, z3, a3, z4, a4, w4, w3, w2, y, x
        )

        w1, b1, w2, b2, w3, b3, w4, b4 = update_params(w1,w2,w3,w4, b1, b2, b3, b4,
            dw1, db1, dw2, db2, dw3, db3, dw4, db4, alpha
        )

        if i % 10 == 0:
            train_preds = np.argmax(a4, axis=0)   # <-- SHOULD BE a4, NOT a2
            train_acc = np.mean(train_preds == y_train) * 100
            print("Iteration:", i, "Training Accuracy:", train_acc, "%")

    return w1, b1, w2, b2, w3, b3, w4, b4


w1, b1, w2, b2, w3, b3, w4, b4 = init_params()

       
    

In [28]:
w1, b1, w2, b2, w3, b3, w4, b4 =  grad_descent(w1, b1, w2, b2, w3, b3, w4, b4, x_train, y_train, 1000 , 0.05)

Iteration: 0 Training Accuracy: 8.51842105263158 %
Iteration: 10 Training Accuracy: 26.75526315789474 %
Iteration: 20 Training Accuracy: 37.373684210526314 %
Iteration: 30 Training Accuracy: 43.77368421052631 %
Iteration: 40 Training Accuracy: 49.15 %
Iteration: 50 Training Accuracy: 53.85263157894736 %
Iteration: 60 Training Accuracy: 57.46052631578947 %
Iteration: 70 Training Accuracy: 60.6 %
Iteration: 80 Training Accuracy: 63.30263157894736 %
Iteration: 90 Training Accuracy: 65.57105263157895 %
Iteration: 100 Training Accuracy: 67.76578947368421 %
Iteration: 110 Training Accuracy: 69.67368421052632 %
Iteration: 120 Training Accuracy: 71.38947368421053 %
Iteration: 130 Training Accuracy: 73.03684210526316 %
Iteration: 140 Training Accuracy: 74.49736842105264 %
Iteration: 150 Training Accuracy: 75.6342105263158 %
Iteration: 160 Training Accuracy: 76.74736842105263 %
Iteration: 170 Training Accuracy: 77.73684210526316 %
Iteration: 180 Training Accuracy: 78.6236842105263 %
Iteration: 1

In [29]:
w1, b1, w2, b2, w3, b3, w4, b4 =  grad_descent(w1, b1, w2, b2, w3, b3, w4, b4, x_train, y_train, 500 , 0.01)

Iteration: 0 Training Accuracy: 91.30526315789473 %
Iteration: 10 Training Accuracy: 91.30789473684212 %
Iteration: 20 Training Accuracy: 91.31315789473685 %
Iteration: 30 Training Accuracy: 91.32894736842105 %
Iteration: 40 Training Accuracy: 91.33421052631579 %
Iteration: 50 Training Accuracy: 91.34473684210526 %
Iteration: 60 Training Accuracy: 91.35263157894737 %
Iteration: 70 Training Accuracy: 91.36315789473684 %
Iteration: 80 Training Accuracy: 91.37105263157895 %
Iteration: 90 Training Accuracy: 91.37368421052632 %
Iteration: 100 Training Accuracy: 91.38157894736842 %
Iteration: 110 Training Accuracy: 91.38421052631578 %
Iteration: 120 Training Accuracy: 91.39473684210526 %
Iteration: 130 Training Accuracy: 91.40789473684211 %
Iteration: 140 Training Accuracy: 91.41052631578947 %
Iteration: 150 Training Accuracy: 91.43157894736842 %
Iteration: 160 Training Accuracy: 91.43947368421053 %
Iteration: 170 Training Accuracy: 91.45263157894736 %
Iteration: 180 Training Accuracy: 91.45

In [30]:
w1, b1, w2, b2, w3, b3, w4, b4 =  grad_descent(w1, b1, w2, b2, w3, b3, w4, b4, x_train, y_train, 500 , 0.05
                                              )

Iteration: 0 Training Accuracy: 91.70526315789473 %
Iteration: 10 Training Accuracy: 91.75263157894736 %
Iteration: 20 Training Accuracy: 91.80526315789473 %
Iteration: 30 Training Accuracy: 91.84210526315789 %
Iteration: 40 Training Accuracy: 91.86315789473684 %
Iteration: 50 Training Accuracy: 91.89473684210526 %
Iteration: 60 Training Accuracy: 91.93684210526317 %
Iteration: 70 Training Accuracy: 91.96842105263158 %
Iteration: 80 Training Accuracy: 91.99473684210527 %
Iteration: 90 Training Accuracy: 92.04473684210527 %
Iteration: 100 Training Accuracy: 92.07105263157894 %
Iteration: 110 Training Accuracy: 92.09736842105262 %
Iteration: 120 Training Accuracy: 92.12894736842105 %
Iteration: 130 Training Accuracy: 92.14473684210527 %
Iteration: 140 Training Accuracy: 92.17368421052632 %
Iteration: 150 Training Accuracy: 92.20789473684209 %
Iteration: 160 Training Accuracy: 92.23421052631579 %
Iteration: 170 Training Accuracy: 92.27368421052631 %
Iteration: 180 Training Accuracy: 92.30

In [31]:
z1, a1, z2, a2, z3, a3, z4, a4 = forward_prop(w1, b1, w2, b2, w3, b3, w4, b4, x_test)

val_predictions = np.argmax(a4, axis=0)
val_accuracy = np.mean(val_predictions == y_test) * 100

print("Validation Accuracy:", val_accuracy, "%")

Validation Accuracy: 92.65 %
